In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
emails = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/input/gcc-pre-processed.csv')
# email_subset = emails[:10000]
#email_subset = emails.sample(frac=0.02, random_state=1)

#print(email_subset.shape)
#print(email_subset.head())
print(emails.shape)
print(emails.head())

(16030, 3)
   Unnamed: 0                                         Message-ID  \
0           0                 <20210401020415.00002c77@tesio.it>   
1           1  <e692f866504485f4b5532b65e5a24cd2@appliantolog...   
2           2                          <xjfczvel97j.fsf@sdf.org>   
3           3  <CAH6eHdSaAqzXWzBwyW8=-OYOHCHye4feYBf-_55ikQ3p...   
4           4  <CAH6eHdS1J3FZ6FaecRX+dYjaj9kJOn=3F5NZH2Sg6kki...   

                                            Content1  
0  Hi Jeff,\r\n\nthanks for fixing your affiliati...  
1  On 2021-03-31 17:04, Giacomo Tesio wrote:\r\n\...  
2  Giacomo Tesio <giacomo@tesio.it> writes:\r\n\n...  
3  On Thu, 1 Apr 2021, 01:05 Giacomo Tesio, <giac...  
4  On Thu, 1 Apr 2021 at 10:13, Jonathan Wakely w...  


In [ ]:
#email_df = pd.DataFrame(parse_into_emails(email_subset.content))
email_df = pd.DataFrame(emails.Content1)
print(email_df.head())

                                            Content1
0  Hi Jeff,\r\n\nthanks for fixing your affiliati...
1  On 2021-03-31 17:04, Giacomo Tesio wrote:\r\n\...
2  Giacomo Tesio <giacomo@tesio.it> writes:\r\n\n...
3  On Thu, 1 Apr 2021, 01:05 Giacomo Tesio, <giac...
4  On Thu, 1 Apr 2021 at 10:13, Jonathan Wakely w...


In [ ]:
email_df.rename(columns = {'Content1':'content'}, inplace = True)

In [ ]:
email_df

,content
0,"Hi Jeff,\r\n\nthanks for fixing your affiliati..."
1,"On 2021-03-31 17:04, Giacomo Tesio wrote:\r\n\..."
2,Giacomo Tesio <giacomo@tesio.it> writes:\r\n\n...
3,"On Thu, 1 Apr 2021, 01:05 Giacomo Tesio, <giac..."
4,"On Thu, 1 Apr 2021 at 10:13, Jonathan Wakely w..."
...,...
16025,"Hi, I have recently been working on issues rel..."
16026,Snapshot gcc-12-20230929 is now available on\r...
16027,\nSent from my iPhone set t\r\n
16028,I have now released the new implementation for...


In [ ]:
import re
import numpy as np

In [ ]:
import gensim

In [ ]:
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

In [ ]:
# spacy for lemmatization
import spacy

In [ ]:
!pip install pyLDAvis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 33.2 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn

In [ ]:
# for plotting
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt

In [ ]:
 import nltk
 nltk.download('stopwords')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# prep NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use','gcc','write','also','work','think','writes','thank','time','get','week','need','see'])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
print(email_df.iloc[2]['content']) # displays info below

Giacomo Tesio <giacomo@tesio.it> writes:

[...]

Hi Giacomo,

thanks for your mail.

Unfortunately I think is clear at this points what's the "Open Source"
corporates position on this affair.

A weak and malleable FSF and Free Software community is to many a very
comfortable situation.  Excluding RMS leveraging cancel culture proved
to be the perfect strategy.  Cancel culture is extremely powerful, it
induces strong emotions and participation on people in good faith,
freedom of speech and rationality go quickly in background (reminds me
something of the past).

Fighting against that often proves to be a desperate task, this is
really just the perfect storm.

Those who loses is just us, a divided and weaker movement.

Regards

  Andrea



/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
email_df

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,content
0,"Hi Jeff,\r\n\nthanks for fixing your affiliati..."
1,"On 2021-03-31 17:04, Giacomo Tesio wrote:\r\n\..."
2,Giacomo Tesio <giacomo@tesio.it> writes:\r\n\n...
3,"On Thu, 1 Apr 2021, 01:05 Giacomo Tesio, <giac..."
4,"On Thu, 1 Apr 2021 at 10:13, Jonathan Wakely w..."
...,...
16025,"Hi, I have recently been working on issues rel..."
16026,Snapshot gcc-12-20230929 is now available on\r...
16027,\nSent from my iPhone set t\r\n
16028,I have now released the new implementation for...


In [ ]:
def clean_email_df(df):
    '''
    These remove symbols and character patterns that don't aid in producing a good summary.
    '''
    #Removing strings related to attatchments and certain non numerical characters.
    patterns = ["\[IMAGE\]","-", "_", "\*", "+","\".\""]
    for pattern in patterns:
        df['content'] = pd.Series(df['content']).str.replace(pattern, "")

    #Remove multiple spaces.
    df['content'] = df['content'].replace('\s+', ' ', regex=True)

    #Blanks are replaced with NaN in the whole dataframe. Then rows with a 'NaN' in the body will be dropped.
    df = df.replace('',np.NaN)
    df = df.dropna(subset=['content'])

    #Remove all Duplicate emails
    #df = df.drop_duplicates(subset='body')
    return df

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<>:6: DeprecationWarning: invalid escape sequence '\['
<>:6: DeprecationWarning: invalid escape sequence '\*'
<>:11: DeprecationWarning: invalid escape sequence '\s'
<>:6: DeprecationWarning: invalid escape sequence '\['
<>:6: DeprecationWarning: invalid escape sequence '\*'
<>:11: DeprecationWarning: invalid escape sequence '\s'
<ipython-input-17-ed22e45a22a5>:6: DeprecationWarning: invalid escape sequence '\['
  patterns = ["\[IMAGE\]","-", "_", "\*", "+","\".\""]
<ipython-input-17-ed22e45a22a5>:6: DeprecationWarning: invalid escape sequence '\*'
  patterns = ["\[IMAGE\]","-", "_", "\*", "+","\".\""]
<ipython-input-17-ed22e45a22a5>:11:

In [ ]:
email_df = clean_email_df(email_df)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<ipython-input-17-ed22e45a22a5>:8: FutureWarning: The default value of regex will change from True to False in a future version.
  df['content'] = pd.Series(df['content']).str.replace(pattern, "")
<ipython-input-17-ed22e45a22a5>:8: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['content'] = pd.Series(df['content']).str.replace(pattern, "")


In [ ]:
email_df

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,content
0,"Hi Jeff, thanks for fixing your affiliation, b..."
1,"On 20210331 17:04, Giacomo Tesio wrote: Not to..."
2,Giacomo Tesio <giacomo@tesio.it> writes: [...]...
3,"On Thu, 1 Apr 2021, 01:05 Giacomo Tesio, <giac..."
4,"On Thu, 1 Apr 2021 at 10:13, Jonathan Wakely w..."
...,...
16025,"Hi, I have recently been working on issues rel..."
16026,Snapshot gcc1220230929 is now available on htt...
16027,Sent from my iPhone set t
16028,I have now released the new implementation for...


In [ ]:
from nltk.tokenize import word_tokenize, sent_tokenize
def remove_stopwords(sen):
    '''
    This function removes stopwords
    '''
    stop_words = stopwords.words('english')
    stop_words.extend(['gnu','fsf','thank','warnings','function','subject',
                       'gcc','thanks','time','week','patch','issue','problem','code','int','welcome',
                       'project','object','branch','email','people','case',
                       'want'])
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new

def tokenize_email(text):
    '''
    This function splits up the body into sentence tokens and removes stop words.
    '''
    clean_sentences = sent_tokenize(text, language='english')
    #removing punctuation, numbers and special characters. Then lowercasing.
    clean_sentences = [re.sub('[^a-zA-Z ]', '',s) for s in clean_sentences]
    clean_sentences = [s.lower() for s in clean_sentences]
    clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]
    return clean_sentences

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
import nltk
nltk.download('punkt')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
#This tokenizing will be the extracted sentences that may be chosen to form the email summaries.
email_df['extractive_sentences'] = email_df['content'].apply(sent_tokenize)
#Splitting the text in emails into cleaned sentences
email_df['tokenized_body'] = email_df['content'].apply(tokenize_email)
#Tokenizing the bodies might have revealed more duplicate emails that should be droped.
email_df = email_df.loc[email_df.astype(str).drop_duplicates(subset='tokenized_body').index]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
email_df

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,content,extractive_sentences,tokenized_body
0,"Hi Jeff, thanks for fixing your affiliation, b...","[Hi Jeff, thanks for fixing your affiliation, ...",[hi jeff fixing affiliation let note doesnt ch...
1,"On 20210331 17:04, Giacomo Tesio wrote: Not to...","[On 20210331 17:04, Giacomo Tesio wrote: Not t...",[giacomo tesio wrote argue counter observation...
2,Giacomo Tesio <giacomo@tesio.it> writes: [...]...,[Giacomo Tesio <giacomo@tesio.it> writes: [......,[giacomo tesio giacomotesioit writes hi giacom...
3,"On Thu, 1 Apr 2021, 01:05 Giacomo Tesio, <giac...","[On Thu, 1 Apr 2021, 01:05 Giacomo Tesio, <gia...",[thu apr giacomo tesio giacomotesioit wrote ev...
4,"On Thu, 1 Apr 2021 at 10:13, Jonathan Wakely w...","[On Thu, 1 Apr 2021 at 10:13, Jonathan Wakely ...",[thu apr jonathan wakely wrote dont think west...
...,...,...,...
15849,"On Mon, 20230904 at 22:13 0400, Eric Feng wrot...","[On Mon, 20230904 at 22:13 0400, Eric Feng wro...","[mon eric feng wrote hi eric, snip know youre ..."
15914,"Am Montag, dem 18.09.2023 um 09:31 0200 schrie...","[Am Montag, dem 18.09.2023 um 09:31 0200 schri...",[montag dem um schrieb richard biener via]
15915,"On Mon, Sep 18, 2023 at 10:17?AM Martin Uecker...","[On Mon, Sep 18, 2023 at 10:17?AM Martin Uecke...",[mon sep martin uecker maueckergmailcom wrote]
15951,"On Sat, Sep 16, 2023 at 10:36:52AM 0200, Marti...","[On Sat, Sep 16, 2023 at 10:36:52AM 0200, Mart...",[sat sep martin uecker wrote right goal migrat...


In [ ]:
# Convert email body to list
data = email_df.content.values.tolist()
#data = email_df.tokenized_body.values.tolist()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# tokenize - break down each sentence into a list of words
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
data_words = list(sent_to_words(data))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
print(data_words[3])

['on', 'thu', 'apr', 'giacomo', 'tesio', 'giacomo', 'tesio', 'it', 'wrote', 'everybody', 'is', 'welcome', 'to', 'send', 'patches', 'for', 'gcc', 'the', 'steering', 'committee', 'doesn', 'decide', 'what', 'people', 'work', 'on', 'and', 'they', 'don', 'approve', 'patches']


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
from gensim.models.phrases import Phrases, Phraser

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Build the bigram and trigram models
bigram = Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = Phrases(bigram[data_words], threshold=100)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = Phraser(bigram)
trigram_mod = Phraser(trigram)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# See trigram example
print(trigram_mod[bigram_mod[data_words[100]]])

['on', 'wed_apr', 'at', 'am_david_malcolm', 'via', 'gcc', 'wrote', 'it', 'reflects', 'the', 'same', 'message', 'that', 'has', 'been', 'sent', 'to', 'new', 'gnu', 'maintainers', 'for', 'the', 'decades', 'the', 'gnu', 'structure', 'and', 'organization', 'document', 'https', 'www', 'gnu', 'org', 'gnu', 'gnustructure', 'en', 'html', 'is', 'basically', 'reflection', 'of', 'that', 'and', 'how', 'we', 'have', 'been', 'doing', 'things', 'for', 'decades', 'we', 've', 'always', 'done', 'it', 'this', 'way', 'is', 'not', 'necessarily', 'good', 'defence', 'of', 'an', 'existing', 'practice', 'you', 'are', 'right', 'the', 'gnu', 'structure', 'document', 'doesn', 'claim', 'to', 'be', 'it', 'just', 'documents', 'the', 'way', 'things', 'are', 'that', 'is', 'true', 'rms', 'appoints', 'which', 'projects', 'become', 'gnu', 'projects', 'or', 'not', 'and', 'who', 'maintains', 'them', 'and', 'as', 'maintainers', 'we', 'have', 'lot', 'of', 'freedom', 'as', 'can', 'be', 'seen', 'here', 'and', 'elsewhere', 'what

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# remove stop_words, make bigrams and lemmatize
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
#python -m spacy download en

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
#nlp = spacy.load('en', disable=['parser', 'ner'])
nlp = spacy.load("en_core_web_sm")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN'])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
print(data_lemmatized[100])

['message', 'maintainer', 'decade', 'structure', 'organization', 'document', 'html', 'reflection', 'thing', 'decade', 'defence', 'practice', 'structure', 'document', 'claim', 'document', 'way', 'thing', 'rm', 'appoint', 'project', 'project', 'maintainer', 'lot', 'freedom', 'sound', 'dictatorship', 'conclusion', 'speak', 'member', 'guy', 'lapse']


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# create dictionary and corpus both are needed for (LDA) topic modeling

# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)
id2word.filter_extremes(no_below=5, no_above=0.5)
# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus_emails = [id2word.doc2bow(text) for text in texts]


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
#!wget http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
#!unzip mallet-2.0.8.zip

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
#mallet_path = './mallet-2.0.8/bin/mallet'

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus_emails,
                                           id2word=id2word,
                                           num_topics=35,
                                           random_state=42,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha=0.01,
                                           per_word_topics=True)

In [ ]:
topics = lda_model.show_topics(num_topics=35, num_words=20, log=False, formatted=False)


In [ ]:
topics

[(0,
  [('warning', 0.28470823),
   ('reference', 0.099236794),
   ('field', 0.0941435),
   ('read', 0.050888494),
   ('mail', 0.0475422),
   ('frame', 0.042854015),
   ('instance', 0.03412493),
   ('web', 0.031302564),
   ('behaviour', 0.025779502),
   ('moment', 0.024196891),
   ('history', 0.02238654),
   ('risk', 0.021095952),
   ('chance', 0.020543618),
   ('follow', 0.016414924),
   ('know', 0.016117152),
   ('lack', 0.015586656),
   ('attack', 0.014116072),
   ('focus', 0.012808814),
   ('violation', 0.011528465),
   ('influence', 0.011379444)]),
 (1,
  [('file', 0.4402076),
   ('mode', 0.12017376),
   ('index', 0.09190854),
   ('name', 0.08461581),
   ('output', 0.08433317),
   ('register', 0.050511986),
   ('auto', 0.039275743),
   ('phase', 0.017696187),
   ('sizeof', 0.0166731),
   ('system', 0.013464058),
   ('opt', 0.011162054),
   ('tue', 0.007741278),
   ('second', 0.0059252735),
   ('gccassert', 0.0041563604),
   ('ensure', 0.0012775052),
   ('return', 0.0001730723),
  

In [ ]:
# Get topics for each email
email_topics = [lda_model.get_document_topics(corpus_email) for corpus_email in corpus_emails]

In [ ]:
email_topics

[[(0, 0.05736723),
  (6, 0.028743438),
  (8, 0.055315327),
  (9, 0.027408196),
  (13, 0.08284461),
  (19, 0.094377175),
  (21, 0.085946605),
  (26, 0.04381563),
  (29, 0.23206426),
  (30, 0.13789055),
  (32, 0.092576005),
  (33, 0.055320587)],
 [(0, 0.038712893),
  (1, 0.018938338),
  (4, 0.018937986),
  (6, 0.019403927),
  (7, 0.027042089),
  (8, 0.056442138),
  (9, 0.018546741),
  (10, 0.018940393),
  (13, 0.056446034),
  (16, 0.09395212),
  (19, 0.16687521),
  (21, 0.057997994),
  (26, 0.025060346),
  (29, 0.10770974),
  (30, 0.15021046),
  (32, 0.102467574),
  (34, 0.018940438)],
 [(0, 0.07038177),
  (7, 0.48697606),
  (16, 0.07038001),
  (25, 0.07038488),
  (29, 0.07191497),
  (31, 0.07038012),
  (33, 0.14006926)],
 [(7, 0.23218371), (21, 0.23218209), (33, 0.46207064)],
 [(7, 0.17709807),
  (8, 0.08898746),
  (11, 0.17710084),
  (16, 0.08898853),
  (21, 0.1771005),
  (26, 0.08897448),
  (30, 0.08897152),
  (31, 0.08898887)],
 [(0, 0.15904814), (10, 0.6315025), (24, 0.15905488)],
 

In [ ]:
num_topics=35
email_keywords = [[word for word, prob in lda_model.show_topic(topic, topn=20)] for topic in range(num_topics)]

In [ ]:
display(email_keywords)

[['warning',
  'reference',
  'field',
  'read',
  'mail',
  'frame',
  'instance',
  'web',
  'behaviour',
  'moment',
  'history',
  'risk',
  'chance',
  'follow',
  'know',
  'lack',
  'attack',
  'focus',
  'violation',
  'influence'],
 ['file',
  'mode',
  'index',
  'name',
  'output',
  'register',
  'auto',
  'phase',
  'sizeof',
  'system',
  'opt',
  'tue',
  'second',
  'gccassert',
  'ensure',
  'return',
  'gate',
  'indentation',
  'whitespace',
  'ebb'],
 ['flag',
  'section',
  'variable',
  'transformation',
  'compile',
  'store',
  'gimple',
  'response',
  'stuff',
  'practice',
  'way',
  'constraint',
  'summary',
  'reading',
  'mention',
  'mailing_list',
  'website',
  'advantage',
  'writing',
  'conclusion'],
 ['support',
  'target',
  'thank',
  'default',
  'lto',
  'avr',
  'debug',
  'way',
  'dwarf',
  'translation_unit',
  'force',
  'inspiration',
  'erick_ochoa',
  'simpleipapass',
  'foobar',
  'item_opindex',
  'mklog_py',
  'subject',
  'workflow'

In [ ]:
for inner_list in email_keywords:
    print(inner_list[:12][:10])

['warning', 'reference', 'field', 'read', 'mail', 'frame', 'instance', 'web', 'behaviour', 'moment']
['file', 'mode', 'index', 'name', 'output', 'register', 'auto', 'phase', 'sizeof', 'system']
['flag', 'section', 'variable', 'transformation', 'compile', 'store', 'gimple', 'response', 'stuff', 'practice']
['support', 'target', 'thank', 'default', 'lto', 'avr', 'debug', 'way', 'dwarf', 'translation_unit']
['change', 'branch', 'comment', 'event', 'release', 'report', 'html', 'structure', 'look', 'form']
['project', 'year', 'resource', 'port', 'programming', 'gsoc', 'compiler_collection', 'talent', 'eric_botcazou', 'pm_gmt']
['type', 'option', 'module', 'argument', 'library', 'datum', 'array', 'name', 'plan', 'access']
['people', 'developer', 'community', 'question', 'service', 'fortran', 'maintainer', 'development', 'experience', 'thing']
['discussion', 'level', 'run', 'copy', 'direction', 'paper', 'sc', 'matter', 'scale', 'involvement']
['sourceware', 'operation', 'extension', 'pipermai

In [ ]:
# topic modeling
# corpus, dictionary and number of topics required for LDA
# alpha and eta are hyperparameters that affect sparsity of the topics
# chunksize is the number of documents to be used in each training chunk
# update_every determines how often the model parameters should be updated
# passes is the total number of training passes
# Print the Keyword in the 10 topics

In [ ]:
print(lda_model.print_topics())# The weights reflect how important a keyword is to that topic.

[(23, '0.330*"code" + 0.125*"system" + 0.106*"behavior" + 0.105*"kind" + 0.077*"order" + 0.065*"language" + 0.064*"approach" + 0.034*"commit" + 0.023*"change" + 0.022*"reply"'), (18, '0.409*"error" + 0.178*"object" + 0.154*"feature" + 0.093*"definition" + 0.039*"bugzilla" + 0.021*"window" + 0.020*"iain" + 0.017*"cause" + 0.016*"wiki" + 0.011*"coroutine"'), (5, '0.551*"project" + 0.188*"year" + 0.091*"resource" + 0.072*"port" + 0.028*"programming" + 0.013*"gsoc" + 0.012*"compiler_collection" + 0.003*"talent" + 0.003*"eric_botcazou" + 0.000*"pm_gmt"'), (28, '0.359*"int" + 0.136*"const" + 0.100*"return" + 0.075*"struct" + 0.071*"tree" + 0.044*"foo" + 0.039*"state" + 0.025*"bool" + 0.024*"index" + 0.020*"void"'), (9, '0.141*"sourceware" + 0.129*"operation" + 0.099*"extension" + 0.068*"pipermail" + 0.058*"goal" + 0.058*"status" + 0.056*"update" + 0.033*"binutil" + 0.031*"topic" + 0.025*"date"'), (8, '0.190*"discussion" + 0.168*"level" + 0.168*"run" + 0.159*"copy" + 0.066*"direction" + 0.055

In [ ]:
doc_lda = lda_model[corpus_emails]

In [ ]:
# Model perplexity and topic coherence provide a convenient
# measure to judge how good a given topic model is.
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus_emails))  # a measure of how good the model is. lower the better.



Perplexity:  -19.78397187934256


In [ ]:
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)



Coherence Score:  0.47141379309232595


In [ ]:
!pip install pandas==1.5.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 24.5 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.1.3
    Uninstalling pandas-2.1.3:
      Successfully uninstalled pandas-2.1.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.
pyldavis 3.4.1 requires pandas>=2.0.0, but you have pandas 1.5.3 which is incompatible.


In [ ]:
# Visualize the topics
pyLDAvis.enable_notebook(sort=True)
vis = pyLDAvis.gensim.prepare(lda_model, corpus_emails, id2word)

In [ ]:
pyLDAvis.display(vis)

In [ ]:
# now using mallet

In [ ]:
#mallet_path = 'mallet-2.0.8/bin/mallet'

In [ ]:
#!pip install gensim==3.8.3

In [ ]:
#from gensim.models.wrappers import LdaMallet
#from gensim.models import LdaMallet

#ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=20, id2word=id2word)
#ldamallet = LdaMallet(mallet_path, corpus=corpus_emails, num_topics=20, id2word=id2word)

In [ ]:
# Show Topics
#print(ldamallet.show_topics(formatted=False))

In [ ]:
# Compute Coherence Score
#coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
#coherence_ldamallet = coherence_model_ldamallet.get_coherence()
#print('\nCoherence Score: ', coherence_ldamallet)

In [ ]:
# # Visualize the topics with mallet model
# pyLDAvis.enable_notebook(sort=True)
# vis = pyLDAvis.gensim.prepare(ldamallet, corpus, id2word)
# pyLDAvis.display(vis)

In [ ]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [ ]:
# run
#model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus_emails, texts=data_lemmatized, start=2, limit=40, step=6)


In [ ]:
# Show graph
#limit=40; start=2; step=6;
#x = range(start, limit, step)
#plt.plot(x, coherence_values)
#plt.xlabel("Num Topics")
#plt.ylabel("Coherence score")
#plt.legend(("coherence_values"), loc='best')
#plt.show()

In [ ]:
# Print the coherence scores
#for m, cv in zip(x, coherence_values):
#    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))

In [ ]:
# Select the model and print the topics
#optimal_model = model_list[4]
#model_topics = optimal_model.show_topics(formatted=False)
#print(optimal_model.print_topics(num_words=10))

In [ ]:
lda_model.show_topic(5)

[('project', 0.5508122),
 ('year', 0.1884936),
 ('resource', 0.09111931),
 ('port', 0.07176185),
 ('programming', 0.02753887),
 ('gsoc', 0.0129073905),
 ('compiler_collection', 0.011677853),
 ('talent', 0.0032495991),
 ('eric_botcazou', 0.0026316904),
 ('pm_gmt', 9.556166e-05)]

In [ ]:
def format_topics_sentences(ldamodel=lda_model, corpus=corpus_emails, texts=data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus_emails]):
        #row = row_list[0] if ldamodel.per_word_topics else row_list
        row = sorted(row[0], key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)



In [ ]:
df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=corpus_emails, texts=data)

Streaming output truncated to the last 5000 lines.
<ipython-input-73-a90a0b9439fd>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
<ipython-input-73-a90a0b9439fd>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
<ipython-input-73-a90a0b9439fd>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
<ipython-input-73-a90a0b9439fd>:14: FutureWarning: The frame.app

In [ ]:
df_topic_sents_keywords

,Dominant_Topic,Perc_Contribution,Topic_Keywords,0
0,29,0.2321,"org, word, page, rm, process, issue, side, swi...","Hi Jeff, thanks for fixing your affiliation, b..."
1,19,0.1669,"move, pm, power, match, reader, style, observa...","On 20210331 17:04, Giacomo Tesio wrote: Not to..."
2,7,0.4870,"people, developer, community, question, servic...",Giacomo Tesio <giacomo@tesio.it> writes: [...]...
3,33,0.4621,"patch, note, implementation, information, than...","On Thu, 1 Apr 2021, 01:05 Giacomo Tesio, <giac..."
4,11,0.1771,"problem, thing, way, text, input, day, precisi...","On Thu, 1 Apr 2021 at 10:13, Jonathan Wakely w..."
...,...,...,...,...
8075,12,0.2444,"function, class, pointer, attribute, code, cal...","On Mon, 20230904 at 22:13 0400, Eric Feng wrot..."
8076,0,0.0286,"warning, reference, field, read, mail, frame, ...","Am Montag, dem 18.09.2023 um 09:31 0200 schrie..."
8077,0,0.0286,"warning, reference, field, read, mail, frame, ...","On Mon, Sep 18, 2023 at 10:17?AM Martin Uecker..."
8078,23,0.2271,"code, system, behavior, kind, order, language,...","On Sat, Sep 16, 2023 at 10:36:52AM 0200, Marti..."


In [ ]:
df_topic_sents_keywords = df_topic_sents_keywords.rename(columns={'0': 'Text'})
df_topic_sents_keywords

,Dominant_Topic,Perc_Contribution,Topic_Keywords,0
0,29,0.2321,"org, word, page, rm, process, issue, side, swi...","Hi Jeff, thanks for fixing your affiliation, b..."
1,19,0.1669,"move, pm, power, match, reader, style, observa...","On 20210331 17:04, Giacomo Tesio wrote: Not to..."
2,7,0.4870,"people, developer, community, question, servic...",Giacomo Tesio <giacomo@tesio.it> writes: [...]...
3,33,0.4621,"patch, note, implementation, information, than...","On Thu, 1 Apr 2021, 01:05 Giacomo Tesio, <giac..."
4,11,0.1771,"problem, thing, way, text, input, day, precisi...","On Thu, 1 Apr 2021 at 10:13, Jonathan Wakely w..."
...,...,...,...,...
8075,12,0.2444,"function, class, pointer, attribute, code, cal...","On Mon, 20230904 at 22:13 0400, Eric Feng wrot..."
8076,0,0.0286,"warning, reference, field, read, mail, frame, ...","Am Montag, dem 18.09.2023 um 09:31 0200 schrie..."
8077,0,0.0286,"warning, reference, field, read, mail, frame, ...","On Mon, Sep 18, 2023 at 10:17?AM Martin Uecker..."
8078,23,0.2271,"code, system, behavior, kind, order, language,...","On Sat, Sep 16, 2023 at 10:36:52AM 0200, Marti..."


In [ ]:
# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

In [ ]:
# Show
df_dominant_topic.head(20)


,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,29,0.2321,"org, word, page, rm, process, issue, side, swi...","Hi Jeff, thanks for fixing your affiliation, b..."
1,1,19,0.1669,"move, pm, power, match, reader, style, observa...","On 20210331 17:04, Giacomo Tesio wrote: Not to..."
2,2,7,0.4870,"people, developer, community, question, servic...",Giacomo Tesio <giacomo@tesio.it> writes: [...]...
3,3,33,0.4621,"patch, note, implementation, information, than...","On Thu, 1 Apr 2021, 01:05 Giacomo Tesio, <giac..."
4,4,11,0.1771,"problem, thing, way, text, input, day, precisi...","On Thu, 1 Apr 2021 at 10:13, Jonathan Wakely w..."
5,5,10,0.6315,"com, https, thought, category, foobar, verifyi...","On Tue, 30 Mar 2021, SAIFI wrote: Gabriele ple..."
6,6,4,0.9364,"change, branch, comment, event, release, repor...",The GCC 10 branch is now frozen in preparatio...
7,7,3,0.3203,"support, target, thank, default, lto, avr, deb...","Hello, just trying one more time. Any directio..."
8,8,4,0.9636,"change, branch, comment, event, release, repor...",Status ====== The GCC 10 branch is frozen for...
9,9,29,0.3676,"org, word, page, rm, process, issue, side, swi...",The first release candidate for GCC 10.3 is a...


In [ ]:
df_dominant_topic.Keywords.iloc[1]

'move, pm, power, match, reader, style, observation, toolchain, way, nature'

In [ ]:
df_dominant_topic.Text.iloc[1]

"On 20210331 17:04, Giacomo Tesio wrote: Not to argue counter to the observation that there is clear bias in terms of large US and EU corporations, but I wonder how that lines up with the affiliations of the major contributors over the recent past to GCC. Quickly eyeballing the output of 'git shortlog s n all releases/gcc9.1.0...master' Seems to show a similar bias in participation. I don't think that's some intentional subversive plot hatched over a decade by bigcorp.com and bigcorp.de to undermine GNU. It reflects the economics of whose willing and able to commit to that work as a full time undertaking. The idea of making the SC more diverse over time, as well as what needs to be done to improve the diversity of the community of active contributors are important. I'm just not sure how it 'moves the needle' on the bias toward a bigcorp.com and bigcorp.de affiliated contributors who are paid to work on GCC full time and the SC necessarily has to represent that constituency of contribut

In [ ]:
df_dominant_topic.Keywords.iloc[36]

'software, term, license, today, control, contributor, contact, contribution, copyright, benefit'

In [ ]:
df_dominant_topic.Text.iloc[10]

'Hi, just a high level question. I know that IPAPTA is a SIMPLEIPAPASS and that ideally it would be better as an IPAPASS. I understand that one of the biggest challenges of changing IPAPTA to an IPAPASS is that on the current LTO framework, the LTRANS stage can happen at the same time for multiple transformations. This means (I think) that if IPAPTA computed the pointsto sets during WPA, during LTRANS there can be new variables, or modified function bodies that therefore no longer correspond to the computed IPAPTA. However, it seems that there\'s a simpler way to change IPAPTA to an IPAPASS. This is by just introducing another section of "allregularipapasses". Something like (not sure if the name inside the brackets has to be unique) INSERTPASSESAFTER (allregularipapasses) // normal stuff TERMINATEPASSLIST (allregularipapasses) INSERTPASSESAFTER (allregularipapasses) NEXTPASS (passipapta); TERMINATEPASSLIST (allregularipapasses) I understand that this still singles out IPAPTA as being 

In [ ]:
# Group top 5 sentences under each topic
sent_topics_sorteddf_mallet = pd.DataFrame()

sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet,
                                             grp.sort_values(['Perc_Contribution'], ascending=[0]).head(1)],
                                            axis=0)

In [ ]:
df_dominant_topic[df_dominant_topic['Dominant_Topic']==23]


,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
42,42,23,0.4841,"code, system, behavior, kind, order, language,...",Even if you consider those who are not scared...
43,43,23,0.2525,"code, system, behavior, kind, order, language,...","Hi Tadeus, On Fri, Apr 2, 2021 at 9:07 AM Tade..."
168,168,23,0.2147,"code, system, behavior, kind, order, language,...","Hi! My name is Carlos Barbosa, and I would lik..."
226,226,23,0.1401,"code, system, behavior, kind, order, language,...","It should remain an acronym, but it should no..."
239,239,23,0.1942,"code, system, behavior, kind, order, language,...","On Sun, 11 Apr 2021, 15:26 Richard Sandiford v..."
...,...,...,...,...,...
7929,7929,23,0.4671,"code, system, behavior, kind, order, language,...","On 6/9/21 2:09 PM, Richard Biener wrote: Sure..."
7954,7954,23,0.2224,"code, system, behavior, kind, order, language,...",Here are the patches I worked on today. It doe...
7996,7996,23,0.1541,"code, system, behavior, kind, order, language,...","Thanks! This is what I get adding , and to me ..."
7998,7998,23,0.0955,"code, system, behavior, kind, order, language,...","Hi Immad, thanks for this patch. Overall, look..."


In [ ]:
topic_23_rows = df_dominant_topic[df_dominant_topic['Dominant_Topic'] == 1]

# Print Keywords for each row
for index, row in topic_23_rows.iterrows():
    print(f"Document Index: {index}, Keywords: {row['Keywords']}")

Document Index: 16, Keywords: file, mode, index, name, output, register, auto, phase, sizeof, system
Document Index: 40, Keywords: file, mode, index, name, output, register, auto, phase, sizeof, system
Document Index: 621, Keywords: file, mode, index, name, output, register, auto, phase, sizeof, system
Document Index: 655, Keywords: file, mode, index, name, output, register, auto, phase, sizeof, system
Document Index: 656, Keywords: file, mode, index, name, output, register, auto, phase, sizeof, system
Document Index: 658, Keywords: file, mode, index, name, output, register, auto, phase, sizeof, system
Document Index: 706, Keywords: file, mode, index, name, output, register, auto, phase, sizeof, system
Document Index: 733, Keywords: file, mode, index, name, output, register, auto, phase, sizeof, system
Document Index: 734, Keywords: file, mode, index, name, output, register, auto, phase, sizeof, system
Document Index: 815, Keywords: file, mode, index, name, output, register, auto, phas

In [ ]:
df_dominant_topic

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,29,0.2321,"org, word, page, rm, process, issue, side, swi...","Hi Jeff, thanks for fixing your affiliation, b..."
1,1,19,0.1669,"move, pm, power, match, reader, style, observa...","On 20210331 17:04, Giacomo Tesio wrote: Not to..."
2,2,7,0.4870,"people, developer, community, question, servic...",Giacomo Tesio <giacomo@tesio.it> writes: [...]...
3,3,33,0.4621,"patch, note, implementation, information, than...","On Thu, 1 Apr 2021, 01:05 Giacomo Tesio, <giac..."
4,4,11,0.1771,"problem, thing, way, text, input, day, precisi...","On Thu, 1 Apr 2021 at 10:13, Jonathan Wakely w..."
...,...,...,...,...,...
8075,8075,12,0.2444,"function, class, pointer, attribute, code, cal...","On Mon, 20230904 at 22:13 0400, Eric Feng wrot..."
8076,8076,0,0.0286,"warning, reference, field, read, mail, frame, ...","Am Montag, dem 18.09.2023 um 09:31 0200 schrie..."
8077,8077,0,0.0286,"warning, reference, field, read, mail, frame, ...","On Mon, Sep 18, 2023 at 10:17?AM Martin Uecker..."
8078,8078,23,0.2271,"code, system, behavior, kind, order, language,...","On Sat, Sep 16, 2023 at 10:36:52AM 0200, Marti..."


In [ ]:
# Reset Index
sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)

In [ ]:
# Format
sent_topics_sorteddf_mallet.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Text"]

In [ ]:
# Show
sent_topics_sorteddf_mallet

,Topic_Num,Topic_Perc_Contrib,Keywords,Text
0,0,0.7481,"warning, reference, field, read, mail, frame, ...","On Mai 26 2021, Uros Bizjak via Gcc wrote: htt..."
1,1,0.7481,"file, mode, index, name, output, register, aut...","On Tue, Jan 10, 2023 at 9:51 AM Paul Koning vi..."
2,2,0.7481,"flag, section, variable, transformation, compi...","On Tue, Oct 04, 2022 at 01:39:04PM 0400, Richa..."
3,3,0.8553,"support, target, thank, default, lto, avr, deb...","On Mon, Sep 18, 2023 at 9:51?AM Alexander Mona..."
4,4,0.9636,"change, branch, comment, event, release, repor...",The GCC 12 branch is now frozen in preparatio...
5,5,0.8553,"project, year, resource, port, programming, gs...",Correct. Very good evaluation. The Gnu Projec...
6,6,0.8985,"type, option, module, argument, library, datum...","On 6/25/23 12:36, Ben Boeckel wrote: But would..."
7,7,0.8553,"people, developer, community, question, servic...","Community, The official glibc IRC channel is n..."
8,8,0.8553,"discussion, level, run, copy, direction, paper...","W dniu 4.07.2023 o?17:55, David Brown pisze: ..."
9,9,0.8553,"sourceware, operation, extension, pipermail, g...","Hi all, Tomorrow's meeting will be canceled, s..."


In [ ]:
#df_dominant_topic.to_csv('/content/drive/MyDrive/Colab Notebooks/data/preproce-dominant_topic.csv', sep=',')

In [ ]:
#sent_topics_sorteddf_mallet.to_csv('/content/drive/MyDrive/Colab Notebooks/data/sorted_df_pre-processed.csv', sep=',')

In [ ]:
#df_dominant_topic['Keywords'] = df_dominant_topic['Keywords'].str.lower().str.split(',')

In [ ]:
df_dominant_topic.head(10)

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,29,0.2321,"org, word, page, rm, process, issue, side, swi...","Hi Jeff, thanks for fixing your affiliation, b..."
1,1,19,0.1669,"move, pm, power, match, reader, style, observa...","On 20210331 17:04, Giacomo Tesio wrote: Not to..."
2,2,7,0.4870,"people, developer, community, question, servic...",Giacomo Tesio <giacomo@tesio.it> writes: [...]...
3,3,33,0.4621,"patch, note, implementation, information, than...","On Thu, 1 Apr 2021, 01:05 Giacomo Tesio, <giac..."
4,4,11,0.1771,"problem, thing, way, text, input, day, precisi...","On Thu, 1 Apr 2021 at 10:13, Jonathan Wakely w..."
5,5,10,0.6315,"com, https, thought, category, foobar, verifyi...","On Tue, 30 Mar 2021, SAIFI wrote: Gabriele ple..."
6,6,4,0.9364,"change, branch, comment, event, release, repor...",The GCC 10 branch is now frozen in preparatio...
7,7,3,0.3203,"support, target, thank, default, lto, avr, deb...","Hello, just trying one more time. Any directio..."
8,8,4,0.9636,"change, branch, comment, event, release, repor...",Status ====== The GCC 10 branch is frozen for...
9,9,29,0.3676,"org, word, page, rm, process, issue, side, swi...",The first release candidate for GCC 10.3 is a...


In [ ]:
df_test = df_dominant_topic
df_test1 = df_dominant_topic

In [ ]:
#df_test['Keywords'] = df_test['Keywords'].apply(lambda keywords: [word for word in keywords])


In [ ]:
df_test['Keywords'].head(10)

0    org, word, page, rm, process, issue, side, swi...
1    move, pm, power, match, reader, style, observa...
2    people, developer, community, question, servic...
3    patch, note, implementation, information, than...
4    problem, thing, way, text, input, day, precisi...
5    com, https, thought, category, foobar, verifyi...
6    change, branch, comment, event, release, repor...
7    support, target, thank, default, lto, avr, deb...
8    change, branch, comment, event, release, repor...
9    org, word, page, rm, process, issue, side, swi...
Name: Keywords, dtype: object

In [ ]:
df_test.Keywords.iloc[33]

'format, issue, optimization, documentation, compilation, performance, host, regression, machine, cost'

In [ ]:
# Function to calculate a score based on the number of matched words
def calculate_score(keywords, user_query_tokens):
   # matching_words = [word for word in user_query_tokens if word in keywords]
    matching_words = [word for word in user_query_tokens if word in keywords]
    return len(matching_words)

In [ ]:
def calculate_jaccard_similarity(keywords, user_query_tokens):
    intersection_size = len(set(keywords) & set(user_query_tokens))
    union_size = len(set(keywords) | set(user_query_tokens))
    jaccard_similarity = intersection_size / union_size if union_size != 0 else 0
    return jaccard_similarity

In [ ]:
#user_query = "variable memory analysis"
user_query = "optimization performance cost"
user_query

'optimization performance cost'

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
# Assuming 'user_query' is the user's query
#from nltk.tokenize import word_tokenize
tokenized_query = [token.lemma_ for token in nlp(user_query.lower())]

In [ ]:
print(tokenized_query)

['optimization', 'performance', 'cost']


In [ ]:
df_dominant_topic.columns

Index(['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords',
       'Text'],
      dtype='object')

In [ ]:
df_dominant_topic['Keywords']

0       org, word, page, rm, process, issue, side, swi...
1       move, pm, power, match, reader, style, observa...
2       people, developer, community, question, servic...
3       patch, note, implementation, information, than...
4       problem, thing, way, text, input, day, precisi...
                              ...                        
8075    function, class, pointer, attribute, code, cal...
8076    warning, reference, field, read, mail, frame, ...
8077    warning, reference, field, read, mail, frame, ...
8078    code, system, behavior, kind, order, language,...
8079    part, unit, reason, place, condition, pattern,...
Name: Keywords, Length: 8080, dtype: object

In [ ]:
df_test['Score'] = df_test['Keywords'].apply(lambda keywords: calculate_score(keywords, tokenized_query))

In [ ]:
df_test1['Score'] = df_test1['Keywords'].apply(lambda keywords: calculate_jaccard_similarity(keywords, tokenized_query))

In [ ]:
df_test.head(10)

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text,Score
0,0,29,0.2321,"org, word, page, rm, process, issue, side, swi...","Hi Jeff, thanks for fixing your affiliation, b...",0
1,1,19,0.1669,"move, pm, power, match, reader, style, observa...","On 20210331 17:04, Giacomo Tesio wrote: Not to...",0
2,2,7,0.4870,"people, developer, community, question, servic...",Giacomo Tesio <giacomo@tesio.it> writes: [...]...,0
3,3,33,0.4621,"patch, note, implementation, information, than...","On Thu, 1 Apr 2021, 01:05 Giacomo Tesio, <giac...",0
4,4,11,0.1771,"problem, thing, way, text, input, day, precisi...","On Thu, 1 Apr 2021 at 10:13, Jonathan Wakely w...",0
5,5,10,0.6315,"com, https, thought, category, foobar, verifyi...","On Tue, 30 Mar 2021, SAIFI wrote: Gabriele ple...",0
6,6,4,0.9364,"change, branch, comment, event, release, repor...",The GCC 10 branch is now frozen in preparatio...,0
7,7,3,0.3203,"support, target, thank, default, lto, avr, deb...","Hello, just trying one more time. Any directio...",0
8,8,4,0.9636,"change, branch, comment, event, release, repor...",Status ====== The GCC 10 branch is frozen for...,0
9,9,29,0.3676,"org, word, page, rm, process, issue, side, swi...",The first release candidate for GCC 10.3 is a...,0


In [ ]:
df_test1.head(20)

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text,Score
0,0,29,0.2321,"org, word, page, rm, process, issue, side, swi...","Hi Jeff, thanks for fixing your affiliation, b...",0
1,1,19,0.1669,"move, pm, power, match, reader, style, observa...","On 20210331 17:04, Giacomo Tesio wrote: Not to...",0
2,2,7,0.4870,"people, developer, community, question, servic...",Giacomo Tesio <giacomo@tesio.it> writes: [...]...,0
3,3,33,0.4621,"patch, note, implementation, information, than...","On Thu, 1 Apr 2021, 01:05 Giacomo Tesio, <giac...",0
4,4,11,0.1771,"problem, thing, way, text, input, day, precisi...","On Thu, 1 Apr 2021 at 10:13, Jonathan Wakely w...",0
5,5,10,0.6315,"com, https, thought, category, foobar, verifyi...","On Tue, 30 Mar 2021, SAIFI wrote: Gabriele ple...",0
6,6,4,0.9364,"change, branch, comment, event, release, repor...",The GCC 10 branch is now frozen in preparatio...,0
7,7,3,0.3203,"support, target, thank, default, lto, avr, deb...","Hello, just trying one more time. Any directio...",0
8,8,4,0.9636,"change, branch, comment, event, release, repor...",Status ====== The GCC 10 branch is frozen for...,0
9,9,29,0.3676,"org, word, page, rm, process, issue, side, swi...",The first release candidate for GCC 10.3 is a...,0


In [ ]:
df_test[df_test['Score'] >= 2 ]

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text,Score
33,33,26,0.6706,"format, issue, optimization, documentation, co...","Dear GCC development team, We've been trying t...",3
74,74,26,0.7481,"format, issue, optimization, documentation, co...","On April 4, 2021 10:26:37 PM GMT02:00, Christo...",3
77,77,26,0.4621,"format, issue, optimization, documentation, co...","On 20210405 3:36 p.m., Jim Wilson wrote:> On S...",3
81,81,26,0.3015,"format, issue, optimization, documentation, co...","On Mon, Apr 5, 2021 at 10:56 PM Simon Marchi v...",3
269,269,26,0.2744,"format, issue, optimization, documentation, co...","On Sun, Apr 11, 2021 at 8:04 AM David Brown <d...",3
...,...,...,...,...,...,...
7472,7472,26,0.4298,"format, issue, optimization, documentation, co...",I would like that contribute technical documen...,3
7660,7660,26,0.2150,"format, issue, optimization, documentation, co...","Hello, Today I have noticed that latest GCC do...",3
7824,7824,26,0.3757,"format, issue, optimization, documentation, co...","Hi, I have been very interested in loop fusion...",3
7877,7877,26,0.1692,"format, issue, optimization, documentation, co...",Hello! You may call me Ilvir. I got an issue w...,3


In [ ]:
df_test1[df_test1['Score'] >= 2 ]

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text,Score
33,33,26,0.6706,"format, issue, optimization, documentation, co...","Dear GCC development team, We've been trying t...",3
74,74,26,0.7481,"format, issue, optimization, documentation, co...","On April 4, 2021 10:26:37 PM GMT02:00, Christo...",3
77,77,26,0.4621,"format, issue, optimization, documentation, co...","On 20210405 3:36 p.m., Jim Wilson wrote:> On S...",3
81,81,26,0.3015,"format, issue, optimization, documentation, co...","On Mon, Apr 5, 2021 at 10:56 PM Simon Marchi v...",3
269,269,26,0.2744,"format, issue, optimization, documentation, co...","On Sun, Apr 11, 2021 at 8:04 AM David Brown <d...",3
...,...,...,...,...,...,...
7472,7472,26,0.4298,"format, issue, optimization, documentation, co...",I would like that contribute technical documen...,3
7660,7660,26,0.2150,"format, issue, optimization, documentation, co...","Hello, Today I have noticed that latest GCC do...",3
7824,7824,26,0.3757,"format, issue, optimization, documentation, co...","Hi, I have been very interested in loop fusion...",3
7877,7877,26,0.1692,"format, issue, optimization, documentation, co...",Hello! You may call me Ilvir. I got an issue w...,3


In [ ]:
relevant_emails = df_test[df_test['Score']>=2]

In [ ]:
relevant_emails

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text,Score
33,33,26,0.6706,"format, issue, optimization, documentation, co...","Dear GCC development team, We've been trying t...",3
74,74,26,0.7481,"format, issue, optimization, documentation, co...","On April 4, 2021 10:26:37 PM GMT02:00, Christo...",3
77,77,26,0.4621,"format, issue, optimization, documentation, co...","On 20210405 3:36 p.m., Jim Wilson wrote:> On S...",3
81,81,26,0.3015,"format, issue, optimization, documentation, co...","On Mon, Apr 5, 2021 at 10:56 PM Simon Marchi v...",3
269,269,26,0.2744,"format, issue, optimization, documentation, co...","On Sun, Apr 11, 2021 at 8:04 AM David Brown <d...",3
...,...,...,...,...,...,...
7472,7472,26,0.4298,"format, issue, optimization, documentation, co...",I would like that contribute technical documen...,3
7660,7660,26,0.2150,"format, issue, optimization, documentation, co...","Hello, Today I have noticed that latest GCC do...",3
7824,7824,26,0.3757,"format, issue, optimization, documentation, co...","Hi, I have been very interested in loop fusion...",3
7877,7877,26,0.1692,"format, issue, optimization, documentation, co...",Hello! You may call me Ilvir. I got an issue w...,3


In [ ]:
len(relevant_emails)

149

In [ ]:
relevant_emails

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text,Score
33,33,26,0.6706,"format, issue, optimization, documentation, co...","Dear GCC development team, We've been trying t...",3
74,74,26,0.7481,"format, issue, optimization, documentation, co...","On April 4, 2021 10:26:37 PM GMT02:00, Christo...",3
77,77,26,0.4621,"format, issue, optimization, documentation, co...","On 20210405 3:36 p.m., Jim Wilson wrote:> On S...",3
81,81,26,0.3015,"format, issue, optimization, documentation, co...","On Mon, Apr 5, 2021 at 10:56 PM Simon Marchi v...",3
269,269,26,0.2744,"format, issue, optimization, documentation, co...","On Sun, Apr 11, 2021 at 8:04 AM David Brown <d...",3
...,...,...,...,...,...,...
7472,7472,26,0.4298,"format, issue, optimization, documentation, co...",I would like that contribute technical documen...,3
7660,7660,26,0.2150,"format, issue, optimization, documentation, co...","Hello, Today I have noticed that latest GCC do...",3
7824,7824,26,0.3757,"format, issue, optimization, documentation, co...","Hi, I have been very interested in loop fusion...",3
7877,7877,26,0.1692,"format, issue, optimization, documentation, co...",Hello! You may call me Ilvir. I got an issue w...,3


In [ ]:
!pip install rank-bm25

In [ ]:
# Assuming relevant_emails is your DataFrame and 'Text' is the column containing lists of strings
#relevant_emails['Combined_Text'] = [' '.join(text_list) for text_list in relevant_emails['Text']]

In [ ]:
#relevant_emails

In [ ]:
from nltk.tokenize import word_tokenize

# Download the necessary resource for tokenization
nltk.download('punkt')
from rank_bm25 import BM25Okapi
# Tokenize and preprocess the 'Text' column
tokenized_text = [word_tokenize(text) for text in relevant_emails['Text']]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
tokenized_text

[['Dear',
  'GCC',
  'development',
  'team',
  ',',
  'We',
  "'ve",
  'been',
  'trying',
  'to',
  'build',
  'reproducibly',
  'the',
  'minimal',
  'NixOS',
  'image',
  ',',
  'and',
  'gcc',
  'was',
  'one',
  'of',
  'the',
  'last',
  'issues',
  'we',
  'had',
  '.',
  'We',
  'found',
  'that',
  'disabling',
  'profiled',
  'bootstrap',
  'compilation',
  'of',
  'GCC',
  'allowed',
  'us',
  'to',
  'get',
  'a',
  'reproducible',
  'build',
  'of',
  'gcc',
  '.',
  'Our',
  'efforts',
  'can',
  'be',
  'followed',
  'here',
  ':',
  'https',
  ':',
  '//github.com/NixOS/nixpkgs/pull/112928',
  'But',
  'I',
  'measured',
  'disabling',
  'this',
  'optimization',
  'to',
  'cost',
  'around',
  '712',
  '%',
  'depending',
  'on',
  'the',
  'build',
  '.',
  'Because',
  'of',
  'this',
  'performance',
  'regression',
  ',',
  'we',
  "'re",
  'trying',
  'to',
  'find',
  'a',
  'middle',
  'ground',
  '.',
  'Ideally',
  'we',
  "'d",
  'like',
  'to',
  'keep',
  

In [ ]:
relevant_emails.Text.iloc[0]

"Dear GCC development team, We've been trying to build reproducibly the minimal NixOS image, and gcc was one of the last issues we had. We found that disabling profiled bootstrap compilation of GCC allowed us to get a reproducible build of gcc. Our efforts can be followed here: https://github.com/NixOS/nixpkgs/pull/112928 But I measured disabling this optimization to cost around 712% depending on the build. Because of this performance regression, we're trying to find a middle ground. Ideally we'd like to keep the performance of gcc as untouched as possible (even if that costs us on compilation time of gcc itself). Compiling gcc twice on the same machine gets us the same output, but compiling on a different architecture gets us a different result. Reading the documentation, it would seem that autoprofiledback bootstrap would use machine metrics and injects them in the build (and we don't use autoprofiledback), But I would not expect the stagetrain of profiledbootstrap to do that. I trie

In [ ]:
#relevant_emails.Combined_Text.iloc[0]

In [ ]:
bm25 = BM25Okapi(tokenized_text)

In [ ]:
# Assuming 'tokenized_query' is the tokenized user query
bm25_scores = bm25.get_scores(tokenized_query)
bm25_scores

array([6.42321923, 0.        , 0.        , 0.        , 4.42407825,
       0.        , 0.        , 0.        , 4.86925936, 0.        ,
       0.        , 0.        , 2.63468427, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 3.29677742,
       4.10730087, 3.46969311, 1.9023977 , 0.        , 0.        ,
       0.        , 0.        , 0.        , 5.36222367, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 2.78426271, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 1.0016963 , 0.        , 0.        ,
       2.85514476, 0.        , 0.        , 2.95945072, 0.        ,
       0.        , 0.        , 0.        , 3.16532157, 0.        ,
       1.52652157, 2.87472871, 0.        , 0.        , 0.        ,
       4.17331656, 3.39118831, 0.        , 2.30497417, 0.        ,
       0.        , 0.        , 2.64072293, 2.09035767, 0.     

In [ ]:
# Combine BM25 scores with relevance scores from LDA
final_scores = [bm25_score + 1 if index in relevant_emails else bm25_score for index, bm25_score in enumerate(bm25_scores)]

In [ ]:
# Rank emails based on final scores
ranked_emails = sorted(range(len(final_scores)), key=lambda i: final_scores[i], reverse=True)

In [ ]:
ranked_emails

[0,
 28,
 8,
 4,
 88,
 65,
 20,
 140,
 21,
 66,
 19,
 123,
 58,
 133,
 53,
 122,
 61,
 50,
 37,
 72,
 12,
 146,
 139,
 86,
 114,
 68,
 126,
 73,
 85,
 104,
 22,
 147,
 148,
 128,
 60,
 131,
 47,
 134,
 1,
 2,
 3,
 5,
 6,
 7,
 9,
 10,
 11,
 13,
 14,
 15,
 16,
 17,
 18,
 23,
 24,
 25,
 26,
 27,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 48,
 49,
 51,
 52,
 54,
 55,
 56,
 57,
 59,
 62,
 63,
 64,
 67,
 69,
 70,
 71,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 87,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 124,
 125,
 127,
 129,
 130,
 132,
 135,
 136,
 137,
 138,
 141,
 142,
 143,
 144,
 145]

In [ ]:
# Print the ranked emails
for i in ranked_emails:
    print(f"Email {i + 1}: Final Score = {final_scores[i]}")

Email 1: Final Score = 6.4232192346051225
Email 29: Final Score = 5.362223669542166
Email 9: Final Score = 4.869259359359619
Email 5: Final Score = 4.424078250319245
Email 89: Final Score = 4.391161320102082
Email 66: Final Score = 4.173316556302594
Email 21: Final Score = 4.107300871868737
Email 141: Final Score = 3.5474803599381737
Email 22: Final Score = 3.469693105665996
Email 67: Final Score = 3.3911883052576854
Email 20: Final Score = 3.296777421639792
Email 124: Final Score = 3.213490445790726
Email 59: Final Score = 3.1653215723339767
Email 134: Final Score = 2.9774327293122385
Email 54: Final Score = 2.9594507239411616
Email 123: Final Score = 2.95932556991518
Email 62: Final Score = 2.874728712613567
Email 51: Final Score = 2.855144755327964
Email 38: Final Score = 2.784262712754876
Email 73: Final Score = 2.6407229335911806
Email 13: Final Score = 2.634684269500021
Email 147: Final Score = 2.6123696680426782
Email 140: Final Score = 2.4983643896685246
Email 87: Final Score =

In [ ]:
# Create a DataFrame with Document_No, Score, and Text
result_df = pd.DataFrame({
    "Document_No": relevant_emails["Document_No"],
    "Score": bm25_scores,
    "Text": relevant_emails["Text"]
})

# Sort the DataFrame by BM25 score in descending order
result_df = result_df.sort_values(by="Score", ascending=False)

# Print the ranked documents
#for rank, row in result_df.iterrows():
#    print(f"Rank {rank + 1}: Score={row['Score']:.4f}, Document: {row['Text']}")

In [ ]:
result_df

,Document_No,Score,Text
33,33,6.423219,"Dear GCC development team, We've been trying t..."
1371,1371,5.362224,I?m working on performance tuning a gcc 8.3 po...
357,357,4.869259,TL;DR Join us for a talk on an ongoing project...
269,269,4.424078,"On Sun, Apr 11, 2021 at 8:04 AM David Brown <d..."
4679,4679,4.391161,"On Fri, May 27, 2022 at 11:52 PM m <m@bitsnbit..."
...,...,...,...
3235,3235,0.000000,"On 9/13/21 20:04, Thomas K?nig wrote: Hello. I..."
1240,1240,0.000000,"On Fri, 20210226 at 04:23 0000, brian.sobulefs..."
3193,3193,0.000000,"On Thu, Sep 9, 2021 at 10:14 AM Aldy Hernandez..."
1337,1337,0.000000,Status ====== GCC trunk which eventually will...


In [ ]:
#result_df.to_csv('/content/drive/MyDrive/Colab Notebooks/data/gcc_result_df_opt.csv', sep=',')

In [ ]:
topn_df=result_df[result_df['Score']>0]

In [ ]:
topn_df

,Document_No,Score,Text
33,33,6.423219,"Dear GCC development team, We've been trying t..."
1371,1371,5.362224,I?m working on performance tuning a gcc 8.3 po...
357,357,4.869259,TL;DR Join us for a talk on an ongoing project...
269,269,4.424078,"On Sun, Apr 11, 2021 at 8:04 AM David Brown <d..."
4679,4679,4.391161,"On Fri, May 27, 2022 at 11:52 PM m <m@bitsnbit..."
3048,3048,4.173317,"Thank you for writing this out Bill! On Fri, O..."
948,948,4.107301,"on 2021/8/30 ??10:11, Bill Schmidt wrote: Hi F..."
6906,6906,3.547480,"Hi everyone, I have a general question regardi..."
949,949,3.469693,"On Tue, Aug 31, 2021 at 11:11 AM Kewen.Lin via..."
3139,3139,3.391188,"edit, added list back in Just to add some AAr..."


In [ ]:
#topn_df.to_csv('/content/drive/MyDrive/Colab Notebooks/data/gcc_topn_df_opt.csv', sep=',')

In [ ]:
relevant_emails

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text,Score
33,33,26,0.6706,"format, issue, optimization, documentation, co...","Dear GCC development team, We've been trying t...",3
74,74,26,0.7481,"format, issue, optimization, documentation, co...","On April 4, 2021 10:26:37 PM GMT02:00, Christo...",3
77,77,26,0.4621,"format, issue, optimization, documentation, co...","On 20210405 3:36 p.m., Jim Wilson wrote:> On S...",3
81,81,26,0.3015,"format, issue, optimization, documentation, co...","On Mon, Apr 5, 2021 at 10:56 PM Simon Marchi v...",3
269,269,26,0.2744,"format, issue, optimization, documentation, co...","On Sun, Apr 11, 2021 at 8:04 AM David Brown <d...",3
...,...,...,...,...,...,...
7472,7472,26,0.4298,"format, issue, optimization, documentation, co...",I would like that contribute technical documen...,3
7660,7660,26,0.2150,"format, issue, optimization, documentation, co...","Hello, Today I have noticed that latest GCC do...",3
7824,7824,26,0.3757,"format, issue, optimization, documentation, co...","Hi, I have been very interested in loop fusion...",3
7877,7877,26,0.1692,"format, issue, optimization, documentation, co...",Hello! You may call me Ilvir. I got an issue w...,3


In [ ]:
ranked_emails

[0,
 28,
 8,
 4,
 88,
 65,
 20,
 140,
 21,
 66,
 19,
 123,
 58,
 133,
 53,
 122,
 61,
 50,
 37,
 72,
 12,
 146,
 139,
 86,
 114,
 68,
 126,
 73,
 85,
 104,
 22,
 147,
 148,
 128,
 60,
 131,
 47,
 134,
 1,
 2,
 3,
 5,
 6,
 7,
 9,
 10,
 11,
 13,
 14,
 15,
 16,
 17,
 18,
 23,
 24,
 25,
 26,
 27,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 48,
 49,
 51,
 52,
 54,
 55,
 56,
 57,
 59,
 62,
 63,
 64,
 67,
 69,
 70,
 71,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 87,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 124,
 125,
 127,
 129,
 130,
 132,
 135,
 136,
 137,
 138,
 141,
 142,
 143,
 144,
 145]

In [ ]:
result_df

,Document_No,Score,Text
33,33,6.423219,"Dear GCC development team, We've been trying t..."
1371,1371,5.362224,I?m working on performance tuning a gcc 8.3 po...
357,357,4.869259,TL;DR Join us for a talk on an ongoing project...
269,269,4.424078,"On Sun, Apr 11, 2021 at 8:04 AM David Brown <d..."
4679,4679,4.391161,"On Fri, May 27, 2022 at 11:52 PM m <m@bitsnbit..."
...,...,...,...
3235,3235,0.000000,"On 9/13/21 20:04, Thomas K?nig wrote: Hello. I..."
1240,1240,0.000000,"On Fri, 20210226 at 04:23 0000, brian.sobulefs..."
3193,3193,0.000000,"On Thu, Sep 9, 2021 at 10:14 AM Aldy Hernandez..."
1337,1337,0.000000,Status ====== GCC trunk which eventually will...
